In [ ]:
%load_ext autoreload

In [1]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, utils


In [2]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

In [3]:
def meshcat_line(x_start, x_end, width):
    x_end_shift = x_end.copy()
    x_end_shift[0:2] += width
    x_start_shift = x_start.copy()
    x_start_shift[1:2] += width
    
    points = np.array([[x_start, x_end, x_end_shift, x_start_shift]]).reshape(-1,3)
    triangles = np.array([[0,1,2],[0,2,3]]).reshape(-1,3)
    mc_geom = meshcat.geometry.TriangularMeshGeometry(points, triangles)
    return mc_geom

In [4]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 50

In [5]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [18]:
#marching cubes
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

In [6]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q2', max=2.0, min=-2.0)

In [7]:
#SPP + IRIS CONFIG
start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])
#
#plot start and target
mat = meshcat.geometry.MeshLambertMaterial(color=0xFFDD36)
mat.reflectivity = 1.0
vis2['start'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['start'].set_transform(
                meshcat.transformations.translation_matrix(start.reshape(-1,)))

mat = meshcat.geometry.MeshLambertMaterial(color=0x06D300)
mat.reflectivity = 1.0
vis2['target'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['target'].set_transform(
                meshcat.transformations.translation_matrix(target.reshape(-1,)))

In [9]:
#using hand-crafted seed points

seed_points = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])    # passing


iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    ellipses.append(ellipse)
    regions.append(hpoly)

Time:   0.18 	Volume:   1.52 	Center: [ 0.19923911 -0.51555715 -0.16204128]
Time:   0.09 	Volume:   0.13 	Center: [ 0.67937999 -0.49625952  1.53735375]
Time:   0.06 	Volume:   1.05 	Center: [-0.06171656 -0.89319338  0.39752367]
Time:   0.10 	Volume:   0.33 	Center: [ 0.05710662 -0.34521315  1.65147226]
Time:   0.05 	Volume:   0.22 	Center: [-0.26468704 -0.33151403  1.54513003]


In [10]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

q_low_np = np.array(q_low)
q_high_np = np.array(q_high)

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 6
seed_points = [start, target]

for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    regions.append(reg)
    ellipses.append(ell)
    
for _ in range(its):
    #rejection sampling to get initial feasible point 
    found = False
    while not found:
        t = np.random.rand(3)
        q_samp = (1-t)*q_low_np + t*q_high_np
        found = (col_func_handle(*q_samp)==0.0)
    print("point found: ", q_samp)
    ell, reg = do_iris(q_samp, iris_options)
    if ell.Volume() < 10.0:
        regions.append(reg)
        ellipses.append(ell)
    

Time:   0.05 	Volume:   0.69 	Center: [-0.18154991 -1.03678815  1.14742869]
Time:   0.07 	Volume:   0.13 	Center: [ 0.64658816 -0.49381788  1.57702836]
point found:  [ 0.03824375 -0.66915857 -0.08962705]
Time:   0.14 	Volume:   1.69 	Center: [-0.05431228 -0.81321199  0.46371192]
point found:  [-0.38569467  0.03181223  0.80732692]
Time:   0.06 	Volume:   0.86 	Center: [ 0.14045421 -0.29618592  0.02269143]
point found:  [ 0.33627599 -0.3242979  -0.87865774]
Time:   0.16 	Volume:   1.51 	Center: [ 0.19096236 -0.5161391  -0.14638354]
point found:  [ 0.13915459 -0.90348558 -0.80327203]
Time:   0.06 	Volume:   1.49 	Center: [-0.02819918 -0.81321197  0.27656914]
point found:  [-0.58903761 -0.64905408  1.29922269]
Time:   0.18 	Volume:   2.10 	Center: [ 0.07399343 -0.66421404 -0.39184949]
point found:  [-0.14943848 -0.04079907  0.71119135]
Time:   0.06 	Volume:   0.82 	Center: [ 0.12709378 -0.2804241   0.08868717]


In [11]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))

for region in regions:
    A = region.A()
    b = region.b()
    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 50, 50, 50, col_hand, 0.5)
    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
    C = ellipses[idx].A()
    d = ellipses[idx].center()
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.15
    vis2['ellipses'][ellstr].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1

In [ ]:
# Solve path planning

start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity([.8, .8, .8])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj = spp.Solve()
print(time.time() - start_time)
print(traj.start_time())
print(traj.end_time())
#visualize
maxit = 60
pts = []
for it in range(maxit):
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    vis2['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.06), mat)
    vis2['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
    
    set_joint_angles(pts[-1].reshape(-1,))
    tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
    R_l2 = tf_l2.rotation()
    tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
    
    tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
    R_la = tf_la.rotation()
    tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
     
    
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['link2']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['link2']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_l2))
    mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
    mat.reflectivity = 1.0
    vis['traj']['linka']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['linka']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_la))
    

In [8]:
#rrt cfg
limits = [np.array(q_low), np.array(q_high)]
mat = meshcat.geometry.MeshLambertMaterial(color= 0x000000 , wireframe=True)

def plotting_fn(parent, child, pos_samp, idx, mat, width):
    vis2['rrt']['tree']['line' + str(idx)].set_object( meshcat_line(parent.pos, child.pos, width), mat)
    
plotting_fn_handle = partial(plotting_fn, mat = mat, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrt_col_fn_handle = partial(collision, col_func_handle = col_func_handle)
    
RRT = rrt.RRT(start = start,
              goal = target,
              limits = limits,
              col_func_handle=rrt_col_fn_handle,
              max_extend_length=0.1,
              extend_steps=0.005,
              init_goal_sample_rate=0.03,
              goal_sample_rate_scaler=0.4,
              verbose = True, 
              plotcallback = plotting_fn_handle)

success, path = RRT.run(4000)
if success:
    mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
    mat.wireframeLinewidth = 2.0
    num_waypoints = len(path)
    for idx in range(num_waypoints-1):
        vis2['rrt']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
    traj= utils.PWLinTraj(path, 5.0)

it: 0 distance to target:  0.901 goalsample prob:  0.439
it: 1 distance to target:  0.901 goalsample prob:  0.439
it: 3 distance to target:  0.806 goalsample prob:  0.477
it: 5 distance to target:  0.711 goalsample prob:  0.515
it: 6 distance to target:  0.680 goalsample prob:  0.528
it: 7 distance to target:  0.585 goalsample prob:  0.566
it: 8 distance to target:  0.555 goalsample prob:  0.578
it: 69 distance to target:  0.539 goalsample prob:  0.584
it: 976 distance to target:  0.516 goalsample prob:  0.594
it: 977 distance to target:  0.421 goalsample prob:  0.632
it: 978 distance to target:  0.326 goalsample prob:  0.670
it: 983 distance to target:  0.231 goalsample prob:  0.708
it: 984 distance to target:  0.136 goalsample prob:  0.746
it: 985 distance to target:  0.041 goalsample prob:  0.784
it: 986 distance to target:  0.001 goalsample prob:  0.800
[RRT] Collision free path found in  986  steps


In [9]:
#loop
substeps = 1000
utils.animate(traj, showres, substeps, 10*substeps)
            

0
1.0           
2.0           
3.0           
4.0           
5.0           
6.0           
7.0           
8.0           
9.0           
100           
110           
120           
130           
140           
150           
160           
170           
180           
190           
200           
210           
220           
230           
240           
250           
260           
270           
280           
290           
300           
310           
320           
330           
340           
350           
360           
370           
380           
390           
400           
410           
420           
430           
440           
450           
460           
470           
480           
490           
500           
510           
520           
530           
540           
550           
560           
570           
580           
590           
600           
610           
620           
630           
640           
650           
660           
670     

566           
567           
568           
569           
570           
571           
572           
573           
574           
575           
576           
577           
578           
579           
580           
581           
582           
583           
584           
585           
586           
587           
588           
589           
590           
591           
592           
593           
594           
595           
596           
597           
598           
599           
600           
601           
602           
603           
604           
605           
606           
607           
608           
609           
610           
611           
612           
613           
614           
615           
616           
617           
618           
619           
620           
621           
622           
623           
624           
625           
626           
627           
628           
629           
630           
631           
632       

874           
873           
872           
871           
870           
869           
868           
867           
866           
865           
864           
863           
862           
861           
860           
859           
858           
857           
856           
855           
854           
853           
852           
851           
850           
849           
848           
847           
846           
845           
844           
843           
842           
841           
840           
839           
838           
837           
836           
835           
834           
833           
832           
831           
830           
829           
828           
827           
826           
825           
824           
823           
822           
821           
820           
819           
818           
817           
816           
815           
814           
813           
812           
811           
810           
809           
808       

319           
318           
317           
316           
315           
314           
313           
312           
311           
310           
309           
308           
307           
306           
305           
304           
303           
302           
301           
300           
299           
298           
297           
296           
295           
294           
293           
292           
291           
290           
289           
288           
287           
286           
285           
284           
283           
282           
281           
280           
279           
278           
277           
276           
275           
274           
273           
272           
271           
270           
269           
268           
267           
266           
265           
264           
263           
262           
261           
260           
259           
258           
257           
256           
255           
254           
253       

339           
340           
341           
342           
343           
344           
345           
346           
347           
348           
349           
350           
351           
352           
353           
354           
355           
356           
357           
358           
359           
360           
361           
362           
363           
364           
365           
366           
367           
368           
369           
370           
371           
372           
373           
374           
375           
376           
377           
378           
379           
380           
381           
382           
383           
384           
385           
386           
387           
388           
389           
390           
391           
392           
393           
394           
395           
396           
397           
398           
399           
400           
401           
402           
403           
404           
405       

...

538           
537           
536           
535           
534           
533           
532           
531           
530           
529           
528           
527           
526           
525           
524           
523           
522           
521           
520           
519           
518           
517           
516           
515           
514           
513           
512           
511           
510           
509           
508           
507           
506           
505           
504           
503           
502           
501           
500           
499           
498           
497           
496           
495           
494           
493           
492           
491           
490           
489           
488           
487           
486           
485           
484           
483           
482           
481           
480           
479           
478           
477           
476           
475           
474           
473           
472       

200           
210           
220           
230           
240           
250           
260           
270           
280           
290           
300           
310           
320           
330           
340           
350           
360           
370           
380           
390           
400           
410           
420           
430           
440           
450           
460           
470           
480           
490           
500           
510           
520           
530           
540           
550           
560           
570           
580           
590           
600           
610           
620           
630           
640           
650           
660           
670           
680           
690           
700           
710           
720           
730           
740           
750           
760           
770           
780           
790           
800           
810           
820           
830           
840           
850           
860       

582           
583           
584           
585           
586           
587           
588           
589           
590           
591           
592           
593           
594           
595           
596           
597           
598           
599           
600           
601           
602           
603           
604           
605           
606           
607           
608           
609           
610           
611           
612           
613           
614           
615           
616           
617           
618           
619           
620           
621           
622           
623           
624           
625           
626           
627           
628           
629           
630           
631           
632           
633           
634           
635           
636           
637           
638           
639           
640           
641           
642           
643           
644           
645           
646           
647           
648       

849           
848           
847           
846           
845           
844           
843           
842           
841           
840           
839           
838           
837           
836           
835           
834           
833           
832           
831           
830           
829           
828           
827           
826           
825           
824           
823           
822           
821           
820           
819           
818           
817           
816           
815           
814           
813           
812           
811           
810           
809           
808           
807           
806           
805           
804           
803           
802           
801           
800           
799           
798           
797           
796           
795           
794           
793           
792           
791           
790           
789           
788           
787           
786           
785           
784           
783       

299           
298           
297           
296           
295           
294           
293           
292           
291           
290           
289           
288           
287           
286           
285           
284           
283           
282           
281           
280           
279           
278           
277           
276           
275           
274           
273           
272           
271           
270           
269           
268           
267           
266           
265           
264           
263           
262           
261           
260           
259           
258           
257           
256           
255           
254           
253           
252           
251           
250           
249           
248           
247           
246           
245           
244           
243           
242           
241           
240           
239           
238           
237           
236           
235           
234           
233       

1.0